# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察填補缺值以及 標準化 / 最小最大化 對數值的影響

# [作業重點]
- 觀察替換不同補缺方式, 對於特徵的影響 (In[4]~In[6], Out[4]~Out[6])
- 觀察替換不同特徵縮放方式, 對於特徵的影響 (In[7]~In[8], Out[7]~Out[8])

In [7]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = '../../Lecture_data/'
df_train  = pd.read_csv(data_path + 'titanic_train.csv')
df_test   = pd.read_csv(data_path + 'titanic_test.csv')

print(df_train.shape)
print(df_test.shape)

train_Y = df_train['Survived']
ids     = df_test['PassengerId']

df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test  = df_test.drop(['PassengerId'] , axis=1)

df = pd.concat([df_train,df_test])
df.head()

(891, 12)
(418, 11)


,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
        
print(f'{len(num_features)} Numeric Features : {num_features}\n')

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [9]:
# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
train_num = train_Y.shape[0]
print(train_num)
df.shape

891


(1309, 5)

# 作業1
* 試著在補空值區塊, 替換並執行兩種以上填補的缺值, 看看何者比較好?

In [42]:
# -1
imputer   = SimpleImputer(strategy = 'constant', fill_value = -1)
df_m1     = imputer.fit_transform(df)
train_X   = df_m1[:train_num]
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('strategy = -1,   ans = %1.8f' %(ans) )

print('--------------------------------')

# mean
imputer   = SimpleImputer(strategy = 'mean')
df_mean   = imputer.fit_transform(df)
train_X   = df_mean[:train_num]
print(train_X.mean())
print(train_Y.mean())
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=10).mean()

print('strategy = mean, ans = %1.8f' %(ans) )

print('--------------------------------')

# most_frequent
imputer   = SimpleImputer(strategy = 'most_frequent')
df_most   = imputer.fit_transform(df)
train_X   = df_most[:train_num]
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('strategy = most, ans = %1.8f' %(ans) )

strategy = -1,   ans = 0.69826448
--------------------------------
13.030545604290193
0.3838383838383838
strategy = mean, ans = 0.70276501
--------------------------------
strategy = most, ans = 0.70267690


In [40]:
df_temp = df[:train_num]

# -1
imputer   = SimpleImputer(strategy = 'constant', fill_value = -1)
train_X   = imputer.fit_transform(df_temp)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('strategy = -1,   ans = %1.8f' %(ans) )

print('--------------------------------')

# mean
imputer   = SimpleImputer(strategy = 'mean')
train_X   = imputer.fit_transform(df_temp)
print(train_X.mean())
print(train_Y.mean())
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=10).mean()

print('strategy = mean, ans = %1.8f' %(ans) )

print('--------------------------------')

# most_frequent
imputer   = SimpleImputer(strategy = 'most_frequent')
train_X   = imputer.fit_transform(df_temp)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('strategy = most, ans = %1.8f' %(ans) )

strategy = -1,   ans = 0.69826448
--------------------------------
13.023313832442067
0.3838383838383838
strategy = mean, ans = 0.70513944
--------------------------------
strategy = most, ans = 0.70267690


In [31]:
print(df.shape)
print(df.mean())

df_temp = df[:train_num]
print(df_temp.shape)
print(df_temp.mean())

(1309, 5)
Pclass     2.294882
Age       29.881138
SibSp      0.498854
Parch      0.385027
Fare      33.295479
dtype: float64
(891, 5)
Pclass     2.308642
Age       29.699118
SibSp      0.523008
Parch      0.381594
Fare      32.204208
dtype: float64


# 作業2
* 使用不同的標準化方式 ( 原值 / 最小最大化 / 標準化 )，搭配羅吉斯迴歸模型，何者效果最好?

In [20]:
df_temp = df[:train_num]

# MinMax
imputer   = SimpleImputer(strategy = 'constant', fill_value = -1)
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (-1, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (-1 ~ 1), ans = %1.8f' %(ans) )

print('--------------------------------')

# MinMax
imputer   = SimpleImputer(strategy = 'constant', fill_value = -1)
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (0, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (0 ~ 1),  ans = %1.8f' %(ans) )

print('--------------------------------')

# Std
imputer   = SimpleImputer(strategy = 'constant', fill_value = -1)
train_X   = imputer.fit_transform(df_temp)
scaler    = StandardScaler()
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = Std,             ans = %1.8f' %(ans) )

print('--------------------------------')

scaler = MinMax (-1 ~ 1), ans = 0.69937552
--------------------------------
scaler = MinMax (0 ~ 1),  ans = 0.70048649
--------------------------------
scaler = Std,             ans = 0.69825820
--------------------------------


In [21]:
df_temp = df[:train_num]

# MinMax
imputer   = SimpleImputer(strategy = 'mean')
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (-1, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (-1 ~ 1), ans = %1.8f' %(ans) )

print('--------------------------------')

# MinMax
imputer   = SimpleImputer(strategy = 'mean')
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (0, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (0 ~ 1),  ans = %1.8f' %(ans) )

print('--------------------------------')

# Std
imputer   = SimpleImputer(strategy = 'mean')
train_X   = imputer.fit_transform(df_temp)
scaler    = StandardScaler()
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = Std,             ans = %1.8f' %(ans) )

print('--------------------------------')

scaler = MinMax (-1 ~ 1), ans = 0.70043585
--------------------------------
scaler = MinMax (0 ~ 1),  ans = 0.70046752
--------------------------------
scaler = Std,             ans = 0.69594140
--------------------------------


In [22]:
df_temp = df[:train_num]

# MinMax
imputer   = SimpleImputer(strategy = 'most_frequent')
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (-1, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (-1 ~ 1), ans = %1.8f' %(ans) )

print('--------------------------------')

# MinMax
imputer   = SimpleImputer(strategy = 'most_frequent')
train_X   = imputer.fit_transform(df_temp)
scaler    = MinMaxScaler(feature_range = (0, 1))
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = MinMax (0 ~ 1),  ans = %1.8f' %(ans) )

print('--------------------------------')

# Std
imputer   = SimpleImputer(strategy = 'most_frequent')
train_X   = imputer.fit_transform(df_temp)
scaler    = StandardScaler()
train_X   = scaler.fit_transform(train_X)
estimator = LogisticRegression(solver = 'lbfgs')
ans = cross_val_score(estimator, train_X, train_Y, cv=5).mean()

print('scaler = Std,             ans = %1.8f' %(ans) )

print('--------------------------------')

scaler = MinMax (-1 ~ 1), ans = 0.70268946
--------------------------------
scaler = MinMax (0 ~ 1),  ans = 0.69935020
--------------------------------
scaler = Std,             ans = 0.70268318
--------------------------------


### In this case, most_frequent + MinMaxScaler (-1 to 1) is best.

In [24]:
a = pd.DataFrame({'x1': [1,2,3], 'x2': [2,4,6]})
b = pd.DataFrame({'x1': [4,5,6], 'x2': [8,10,12]})

c = pd.concat([a, b], keys=['s1', 's2'])
print(c)
print(c.mean())

      x1  x2
s1 0   1   2
   1   2   4
   2   3   6
s2 0   4   8
   1   5  10
   2   6  12
x1    3.5
x2    7.0
dtype: float64
